<a href="https://colab.research.google.com/github/ashwanijha8068/btp_dashboard/blob/main/btp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from transformers import BertTokenizer, BertForSequenceClassification, AdamW,get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch.nn.functional as F


data = pd.read_csv('cybertweets.csv', encoding='utf-8', error_bad_lines=False)
data.head()
np.random.seed(42)
# Selecting the feature and target
X = data['clean_tweet']
y = data['relevant']

# # Vectorizing the 'clean_tweet' column using TF-IDF
# vectorizer = TfidfVectorizer()
# X_vectorized = vectorizer.fit_transform(X)
#print(X_vectorized)
#Splitting the dataset into training and test sets (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# # #Tokenize the tweets
# # tokenizer = TweetTokenizer()
# # data['tokenized_tweets'] = data['clean_tweet'].apply(tokenizer.tokenize)
# # model_w2v = Word2Vec(sentences=data['tokenized_tweets'], vector_size=100, window=5, min_count=1, workers=4)
# # w2v_vectors = np.array([np.mean([model_w2v.wv[token] for token in tokens if token in model_w2v.wv] or [np.zeros(100)], axis=0)
# #                         for tokens in data['tokenized_tweets']])

# # #Splitting the dataset for Word2Vec
# # X_train, X_test, y_train, y_test = train_test_split(w2v_vectors, data['relevant'], test_size=0.2, random_state=42)





# # Initialize the RandomForest Classifier
# rf_classifier = RandomForestClassifier(n_estimators=100)

# # Train the classifier
# rf_classifier.fit(X_train, y_train)

# # Predict on the test set
# rf_predictions = rf_classifier.predict(X_test)

# # Calculate the accuracy
# rf_accuracy = accuracy_score(y_test, rf_predictions)


# # Initialize the Logistic Regression model
# log_reg = LogisticRegression(random_state=42)

# # Train the model
# log_reg.fit(X_train, y_train)

# # Predict on the test set
# log_reg_predictions = log_reg.predict(X_test)

# # Calculate the accuracy
# log_reg_accuracy = accuracy_score(y_test, log_reg_predictions)
# print('RandomForest Accuracy:', rf_accuracy)
# print('Logistic Regression Accuracy:', log_reg_accuracy)

# # Initialize the classifiers
# knn_classifier = KNeighborsClassifier()
# svc_classifier = SVC(random_state=42)
# mnb_classifier = MultinomialNB()
# dt_classifier = DecisionTreeClassifier(random_state=42)

# # Train and predict with KNN
# knn_classifier.fit(X_train, y_train)
# knn_predictions = knn_classifier.predict(X_test)
# knn_accuracy = accuracy_score(y_test, knn_predictions)
# print('KNN Accuracy:', knn_accuracy)

# # Train and predict with SVC
# svc_classifier.fit(X_train, y_train)
# svc_predictions = svc_classifier.predict(X_test)
# svc_accuracy = accuracy_score(y_test, svc_predictions)
# print('SVC Accuracy:', svc_accuracy)

# #Train and predict with Multinomial Naive Bayes
# scaler = MinMaxScaler()
# w2v_vectors_scaled = scaler.fit_transform(w2v_vectors)

# # Now use w2v_vectors_scaled for training and prediction
# X_train_nb, X_test_nb, y_train_nb, y_test_nb = train_test_split(w2v_vectors_scaled, data['relevant'], test_size=0.2, random_state=42)
# mnb_classifier.fit(X_train_nb, y_train_nb)
# mnb_predictions = mnb_classifier.predict(X_test_nb)
# mnb_accuracy = accuracy_score(y_test_nb, mnb_predictions)
# print('Multinomial Naive Bayes Accuracy:', mnb_accuracy)

# # Train and predict with Decision Tree
# dt_classifier.fit(X_train, y_train)
# dt_predictions = dt_classifier.predict(X_test)
# dt_accuracy = accuracy_score(y_test, dt_predictions)
# print('Decision Tree Accuracy:', dt_accuracy)

<ipython-input-1-a7ec7bf98d8b>:24: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('cybertweets.csv', encoding='utf-8', error_bad_lines=False)


In [ ]:
#Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_1 = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize the input data
X_train_tokens = tokenizer(list(X_train), padding=True, truncation=True, return_tensors='pt')
X_test_tokens = tokenizer(list(X_test), padding=True, truncation=True, return_tensors='pt')

# Create DataLoader for training and test sets
train_dataset = TensorDataset(X_train_tokens['input_ids'], X_train_tokens['attention_mask'], torch.tensor(y_train.values))
test_dataset = TensorDataset(X_test_tokens['input_ids'], X_test_tokens['attention_mask'], torch.tensor(y_test.values))

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Define optimizer and loss function
optimizer = AdamW(model_1.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1.to(device)

for epoch in range(epochs):
    model_1.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}'):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_1(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_loss}')

# Evaluate on the test set
model_1.eval()
predictions = []
true_labels = []

for batch in tqdm(test_dataloader, desc='Evaluating'):
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model_1(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate accuracy on the test set
accuracy = accuracy_score(true_labels, predictions)

print(f'Test Accuracy: {accuracy}')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch 2/3, Average Training Loss: 0.10646845117352619


Epoch 3/3: 100%|██████████| 3128/3128 [06:05<00:00,  8.56it/s]


Epoch 3/3, Average Training Loss: 0.07114767527185849


Evaluating: 100%|██████████| 783/783 [00:27<00:00, 28.75it/s]

Test Accuracy: 0.9571679718715039


In [ ]:
#second model
# Step 1: Read the CSV file
df = pd.read_csv('cybertweets_annotated.csv', encoding='utf-8')

# Step 2: Filter out 'irrelevant' annotations
df_filtered = df[df['annotation'].isin(['threat', 'business'])]
print(len(df_filtered))

# Step 3: Split the dataset into training and testing sets
train_df, test_df = train_test_split(df_filtered, test_size=0.2, random_state=42)

# Step 4: Prepare the data for BERT model training

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text and ensure that the tokenizer returns PyTorch tensors
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True, return_tensors='pt')

# Convert labels to tensors
train_labels = torch.tensor(train_df['annotation'].map({'business': 0, 'threat': 1}).values)
test_labels = torch.tensor(test_df['annotation'].map({'business': 0, 'threat': 1}).values)

# Create a TensorDataset
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

# Create DataLoaders
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=8)
test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=8)



print("Data prepared for BERT model training.")

# Step 5: Build a BERT model for text classification
model_2 = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Step 6: Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_2.to(device)

optimizer = AdamW(model_2.parameters(), lr=2e-5)

# Total number of training steps
total_steps = len(train_loader) * 3 # 3 epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(3):  # Loop over the dataset multiple times
    model_2.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # Add batch to GPU
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model_2(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Update progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

print('Training complete.')

10611
Data prepared for BERT model training.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 2: 100%|██████████| 1061/1061 [07:09<00:00,  2.47it/s, loss=0.0261]

Training complete.


In [ ]:
# Step 7: Evaluate the model
model_2.eval()
predictions, true_labels = [], []

for batch in tqdm(test_loader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model_2(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    predictions.append(logits)
    true_labels.append(label_ids)

# Calculate the accuracy of the model
predictions = np.argmax(np.concatenate(predictions, axis=0), axis=1)
true_labels = np.concatenate(true_labels, axis=0)
accuracy = np.sum(predictions == true_labels) / len(true_labels)
print(f"Model accuracy on the test set: {accuracy:.4f}")
print(len(predictions))

100%|██████████| 266/266 [00:19<00:00, 13.84it/s]

Model accuracy on the test set: 0.8417
2123


In [ ]:


# Load the new CSV file
new_df = pd.read_csv('cybertweets.csv', encoding='utf-8')

# Filter out rows where the 'relevant' column value is 0
new_df_filtered = new_df[new_df['relevant'] != 0]

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the 'clean_tweet' column
new_encodings = tokenizer(new_df_filtered['clean_tweet'].tolist(), truncation=True, padding=True, return_tensors='pt')

# Create a TensorDataset
new_dataset = TensorDataset(new_encodings['input_ids'], new_encodings['attention_mask'])

# Create a DataLoader
new_loader = DataLoader(new_dataset, batch_size=8)

# Prediction on new data
model_2.eval()
predictions = []

for batch in new_loader:
    # Add batch to GPU or CPU
    b_input_ids, b_input_mask = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        outputs = model_2(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    batch_predictions = np.argmax(logits, axis=1)
    predictions.extend(batch_predictions)


new_df_filtered['prediction'] = predictions


new_df_filtered.to_csv('cybertweets_predictions.csv', index=False)


<ipython-input-6-b69da9e062f2>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_filtered['prediction'] = predictions


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the save paths for the models
model_save_path_1 = "/content/drive/My Drive/Model_1"
model_save_path_2 = "/content/drive/My Drive/Model_2"

# Save the models
model_1.save_pretrained(model_save_path_1)
model_2.save_pretrained(model_save_path_2)


MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install fastapi uvicorn python-multipart


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
import pandas as pd
from io import BytesIO

app = FastAPI()
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], # Adjust this to your needs
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


# Load tokenizer and models
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_1 = BertForSequenceClassification.from_pretrained(model_save_path_1)
model_2 = BertForSequenceClassification.from_pretrained(model_save_path_2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1.to(device)
model_2.to(device)

class PredictionInput(BaseModel):
    model_id: int

@app.post("/predict/")
async def predict(file: UploadFile = File(...), input: PredictionInput = None):
    # Read the file into a DataFrame
    contents = await file.read()
    df = pd.read_csv(BytesIO(contents))

    # Choose the model based on the input
    model = model_1 if input.model_id == 1 else model_2

    # Preprocess and predict
    input_ids = tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
    input_ids = input_ids.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).cpu().numpy()

    # Return the predictions
    return {"predictions": predictions}


In [ ]:
!pip install fastapi uvicorn pyngrok nest-asyncio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 730.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
